In [ ]:
from copy import deepcopy

import numpy as np
import pandas as pd

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
CONFIGS = {
    'data_path': '../data/',
    'model_path': '../model/',
    'model_name': 'super_basic',
    
    'test_lenght': 24,
    'valid_start_index': 1992,
    'test_start_index': 2016,
    
    'batch_size': 64,
    'learning_rate': 1e-4,
    'epochs': 100,
    'es_patience': 10,
    
    'window_size': 7*24,
    'target_length': 3,
}

In [ ]:
data_path = '../data/'

train_origin = pd.read_csv(data_path+'train.csv', encoding='cp949')

In [ ]:
data = deepcopy(train_origin)

data.columns = [
    'num', 'date_time', 'target', 'temp', 'wind',
    'humid', 'rain', 'sun', 'non_elec_eq', 'sunlight_eq'
]

data = data.loc[data['num'] == 1, ['target']]

print(f'data.shape: {data.shape}')